In this dataset we will analyze Sacramento police vehicle stop data. Our goal of this data project is to analyze stop data from Sacramento, specifically focusing what types of people and groups are pulled over, and later on, to see if we can predict, based on interpretable features, whether a vehicle being stopped by the police will result in an arrest or citation.

# Step 1: Getting the data

Fortunately, the city of Sacramento has a lot of great data on its website, including police vehicle stop data. Living in the Sacramento area, it will be cool to analyze data and understand more about stops work in a city close by me. 
The link for the data is below. <br>

http://data.cityofsacramento.org/datasets/a8cb4c137c824e939dca586c6dc77da9_0 <br>
    
I have downloaded the data in csv format.
    
    

# Step 2: Understanding the Data

In [5]:
import pandas as pd
import numpy as np
import pytz
import warnings
warnings.filterwarnings('ignore')

In [6]:
def stop_data():
    stopdata = pd.read_csv("Sacramento_Police_Vehicle_Stop_Data.csv")
    return stopdata

In [7]:
stopdata = pd.read_csv("Sacramento_Police_Vehicle_Stop_Data.csv")
stopdata.head(5)

,Idx,CitationNumber,EntryDate,StopType,StopDTS,StopCause,ViolationNumber,SpecialCondition,Race,Gender,...,ResultCitation,ResultArrest,ResultReport,ResultAdvised,ResultFC,CadCallNumber,DetentionDuration,OfficerAbra,LimitedLanguage,FID
0,437796,NaN,2017-07-06T05:04:02.000Z,VEHICLE,2017-07-06T04:58:00.000Z,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,22108 VC,NOT APPLICABLE,ASIAN/PACIFIC ISLANDER,F,...,False,False,False,True,False,17-190339,4,4425,False,2001
1,437797,NaN,2017-07-06T05:46:55.000Z,VEHICLE,2017-07-06T05:41:00.000Z,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,22450,NOT APPLICABLE,AFRICAN AMERICAN,F,...,False,False,False,True,False,17-190380,15,4294,False,2002
2,437798,NaN,2017-07-06T06:05:59.000Z,VEHICLE,2017-07-06T05:57:00.000Z,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,24616,NOT APPLICABLE,AFRICAN AMERICAN,M,...,False,False,False,True,False,17-190396,10,4411,False,2003
3,437799,NaN,2017-07-06T06:06:16.000Z,VEHICLE,2017-07-06T05:13:00.000Z,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,4000A,NOT APPLICABLE,AFRICAN AMERICAN,F,...,False,False,False,True,False,17-190348,5,4444,False,2004
4,437800,NaN,2017-07-06T06:15:39.000Z,VEHICLE,2017-07-06T06:02:00.000Z,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,24600,NOT APPLICABLE,AFRICAN AMERICAN,M,...,False,False,False,True,False,17-190402,5,4360,False,2005


Perfect! We have our data loaded into a pandas dataframe! Lets try to understand our features at the data we are working with. The link below explains some of the field in data field.

https://www.cityofsacramento.org/-/media/Corporate/Files/Police/Transparency/VSDF/Understanding-Vehicle-Stop-Data-Fields-and-Points-of-Collection.pdf?la=en

Lets go through the data column by column (there are 36), some columns we might already would understand (idx, EntryDate, etc), however some we might need to understand more.


In [8]:
print(stopdata.columns)
print(len(stopdata.columns))

Index(['Idx', 'CitationNumber', 'EntryDate', 'StopType', 'StopDTS',
       'StopCause', 'ViolationNumber', 'SpecialCondition', 'Race', 'Gender',
       'BirthYear', 'ExitVehicle', 'PassengerCount', 'SearchDriver',
       'SearchPassenger', 'SearchVehicle', 'Authority', 'ConsentRequested',
       'ConsentGranted', 'FSWeapons', 'FSDrugs', 'FSCash', 'FSVehicle',
       'FSAlcohol', 'FSOtherProperty', 'FSNothing', 'ResultCitation',
       'ResultArrest', 'ResultReport', 'ResultAdvised', 'ResultFC',
       'CadCallNumber', 'DetentionDuration', 'OfficerAbra', 'LimitedLanguage',
       'FID'],
      dtype='object')
36


In [9]:
stopdata['StopType'].unique()

array(['VEHICLE'], dtype=object)

In [10]:
stopdata['StopCause'].value_counts()

REGISTRATION OR REQUIRED EQUIPMENT VIOLATION     23163
MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE    18983
NOT APPLICABLE                                    4033
STATE CODE VIOLATION                              3561
LOCAL CODE VIOLATION                               134
Name: StopCause, dtype: int64

In [11]:
stopdata['SpecialCondition'].unique()

array(['NOT APPLICABLE', 'PRE-EXISTING KNOWLEDGE/INFORMATION',
       'VEHICLE INSPECTION', 'CALL FOR SERVICE', 'SPECIAL DETAIL',
       'DUI CHECKPOINT', 'BOLO'], dtype=object)

In [12]:
stopdata['ExitVehicle'].value_counts()

NO     41500
YES     7756
Name: ExitVehicle, dtype: int64

In [13]:
stopdata['PassengerCount'].value_counts()

0      31046
1      12568
2       3965
3       1671
4        497
5         83
6         20
10        10
7          4
8          3
15         2
30         1
25         1
20         1
425        1
9          1
Name: PassengerCount, dtype: int64

In [14]:
stopdata['Authority'].value_counts()

NOT APPLICABLE        43888
PAROLE/PROBATION       4236
PROBABLE CAUSE          607
TOW INVENTORY           418
TERRY CURSORY           393
INCIDENT TO ARRE        311
INCIDENT TO ARREST       21
Name: Authority, dtype: int64

### Columns: 

<b>Idx:</b> index of a certain stop, our primary key <br>
<b>CitationNumber: </b>The number of a citation; looking through the data, contains null if no citation, or string of citation number <br>
<b>EntryDate: </b>When the stop was entered into the database (in UTC) <br>
<b>StopType: </b> The only value for this column is "VEHICLE", so it is essentially useless (see code above) <br>
<b>StopDTS: </b>This is the time the vehicle was stopped (in UTC) <br>
<b>StopCause: </b> Probable Cause For Stop <br>
<b>ViolationNumber: </b>The exact violation that the driver commited <br>
<b>SpecialCondition: </b> This adds information to StopCause; for example if the stop was a result of a DUI checkpoint or a call for inspection <br>
<b>Race: </b>Race of driver stopped (ASIAN, WHITE, etc.) <br>
<b>Gender: </b>Gender of driver stopped (F, M) <br>
<b>BirthYear: </b>birth year of driver <br>
<b>ExitVehicle: </b>Binary Variable, indicates whether driver had to exit vehicle <br>
<b>PassengerCount: </b>Number of passengers in the vehicle (not including driver)  <br>
<b>SearchDriver: </b>binary variable, indicates if driver was searched <br>
<b>SearchPassenger: </b>binary variable, indicates if passenger was searched<br>
<b>SearchVehicle: </b>binary variable, indicates if vehicle was searched<br>
<b>ConsentRequested: </b> binary variable, whether or not officer requested to search vehicle ? <br>
<b>ConsentGranted: </b>binary variable, whether or not driver granted request to search vehicle ? <br>
<b> CadCallNumber: </b> Report number, this should be unique <br>
<b> DetentionDuration: </b> Amount of time spent during the stop, in minutes </b> <br>
<b> LimitedLanguage: </b> Binary variable, indicates if driver had limited language <br>
<b> FID: </b> Unique Identifier </b> <br>
<b> FS columns (Weapons, alchohol, etc): </b> If search was done, what was discovered/seized? <br>
<b> Result columns (Citation, Arrest, Advised, etc): </b>Result <br>
<b> OfficerAbra: </b> Unique Officer Identifier <br>
<b>Authority: </b>Primary Search Authority <br>



 



# Step 3: Cleaning the Data

Because most likely we will have to run cells a lot to manipulate values in each column- the code needs to be reusable- it is best to write our cleaning data as functions and then later add all our functions to a data pipeline before we export the file back as a csv. From a software engineering perspective, this means abstracting parts of our code into reusable functions/

In [15]:
### check for columns where we will have to deal with missing data
for i in stopdata.columns:
    print(i + " : "  + str(stopdata[i].isna().sum()) )

Idx : 0
CitationNumber : 32880
EntryDate : 0
StopType : 0
StopDTS : 0
StopCause : 0
ViolationNumber : 81
SpecialCondition : 0
Race : 0
Gender : 0
BirthYear : 0
ExitVehicle : 618
PassengerCount : 0
SearchDriver : 0
SearchPassenger : 0
SearchVehicle : 0
Authority : 0
ConsentRequested : 24223
ConsentGranted : 46734
FSWeapons : 0
FSDrugs : 0
FSCash : 0
FSVehicle : 0
FSAlcohol : 0
FSOtherProperty : 0
FSNothing : 0
ResultCitation : 0
ResultArrest : 0
ResultReport : 0
ResultAdvised : 0
ResultFC : 0
CadCallNumber : 0
DetentionDuration : 0
OfficerAbra : 0
LimitedLanguage : 0
FID : 0


Lets go through each column and see what data might need to be cleaned. This will be a long, tedious process, so enjoy the process.

### CitationNumber

As we found out earlier, citation numbers contain a lot of NANs because the majority of people don't get citations. Because we don't care about the CitationNumber, but rather if someone got a citation or not, lets change this column name to Citation, and map NANs to False and strings to True.

In [16]:
def citationnumberfix(data):
    data.rename(columns={'CitationNumber': 'Citation'}, inplace=True)
    data['Citation'].fillna(False, inplace=True)
    data.loc[data['Citation'] != 0, 'Citation'] = True
    return data

In [17]:
stopdata = citationnumberfix(stopdata)

In [18]:
stopdata[~stopdata['Citation'].isna()]['ResultCitation'].value_counts()

False    31415
True     18459
Name: ResultCitation, dtype: int64

The code above shows there is a discrepancy between CitationNumber and ResultCitation - we would expect when ResultCitation to be True, CitationNumber would always be filled, however, this is not always the case. More research needs to go into how these metrics were attained.

### Dates and StopType

Because we found the StopType earlier to be useless (only one value), we will drop this column as well (later on). We will convert our date columns to more interpretable variables ; month, year, day, day of week, and hour

In [19]:
stopdata['StopDTS'].head(5)

0    2017-07-06T04:58:00.000Z
1    2017-07-06T05:41:00.000Z
2    2017-07-06T05:57:00.000Z
3    2017-07-06T05:13:00.000Z
4    2017-07-06T06:02:00.000Z
Name: StopDTS, dtype: object

In [20]:
def convertdatecolumns(data, column):
    data[column] = pd.to_datetime(data[column])  
    newstop = pd.DataFrame(data[column])
    newstop.set_index(data[column], inplace=True)
    newstop.index = newstop.index.tz_convert('America/Los_Angeles')
    newstop['hour'] = newstop.index.hour
    newcol = 'Stop'
    if column == 'EntryDate': newcol = 'Entry'
    data.loc[:, newcol + 'Month'] = data[column].dt.month
    data.loc[:, newcol + 'Year'] = data[column].dt.year
    data.loc[:, newcol + 'DayofMonth'] = data[column].dt.day
    data.loc[:, newcol + 'DayofWeek'] = data[column].dt.dayofweek
    data.loc[:, newcol + 'Hour'] = newstop.index.hour
    return data

In [21]:
stopdata = convertdatecolumns(stopdata, 'StopDTS')
stopdata = convertdatecolumns(stopdata, 'EntryDate')

### StopCause

In [22]:
stopdata['StopCause'].unique()

array(['MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE',
       'REGISTRATION OR REQUIRED EQUIPMENT VIOLATION',
       'LOCAL CODE VIOLATION', 'STATE CODE VIOLATION', 'NOT APPLICABLE'],
      dtype=object)

In [23]:
stopdata['StopCause'].value_counts()

REGISTRATION OR REQUIRED EQUIPMENT VIOLATION     23163
MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE    18983
NOT APPLICABLE                                    4033
STATE CODE VIOLATION                              3561
LOCAL CODE VIOLATION                               134
Name: StopCause, dtype: int64

All good here, it looks like no cleaning needs to be done for this column.

### Violation Number

In [24]:
stopdata['ViolationNumber'].isna().sum()

81

In [25]:
stopdata['ViolationNumber'].value_counts()[:10]

4000A       5265
5200        3854
23123       3624
22450       3507
CVC         3208
22350       2936
4000        1357
5200 VC     1274
23123.5     1145
4000A VC     917
Name: ViolationNumber, dtype: int64

Right away, we notice some major problems with this feature; - for example "4000A" and "4000A VC" and "4000 CVC" mean the same thing. In addition, there are 81 Nan values. Let's try to "standardize" the data as best as we can, for simplicity we will only consider violations by there number, aka treating "4000A" the same as "4000B" - both these would be mapped to 4000.

In [26]:
def violationnumberfix(data):
    data.loc[data['ViolationNumber'].isna(), 'ViolationNumber'] = 'UNKNOWN'
    data['ViolationNumber'] = data['ViolationNumber'].str.replace("VC", "").str.replace(" ", "").str.replace("C", "").str.replace("A", "").str.replace("(", "").str.replace(")", "").str.replace("B", "")
    data.loc[data['ViolationNumber'] == "", 'ViolationNumber'] = 'UNKNOWN'
    replace_dict = {"4000A": "4000", 
                "4000(A)": "4000", 
                "4000AC": "4000", 
                "4000A1": "4000", 
                "4000(A)(1)": "4000",
                "4000(A)C": "4000",
                "5200A": "5200",
                "23123.5": "23123",
                "STOPSIGN": "22450",
                "SPEED": "22350",
                "BRKELIGHT": "24603",
                "WINDOWTINT": "26708",
                "NOPLTES": "5200",
                "REDLIGHT": "21453",
                "SPEEDING": "22350",
                "ELLPHONE": "23123",
                "TURNSIGNL": "22107",
                "EXPREG": "4000",
                "RKELIGHT": "24252"}

    data['ViolationNumber'].replace(replace_dict, inplace=True)
    return data

In [27]:
stopdata = violationnumberfix(stopdata)
stopdata['ViolationNumber'].value_counts()[:15]

4000       9201
5200       6694
23123      5140
22450      4713
22350      3924
UNKNOWN    3360
24252      1461
21453      1266
22107       959
24400       766
24603       749
26708       677
40001       648
24600       546
24601       477
Name: ViolationNumber, dtype: int64

For know we have corrected as much as we can, it will take another few hours to clean every single value in this column or to think of an algorithm to fix this; if this model was going into production, it most certainly would be worth it to comb through all the data and fix it, however for the purposes of this project, we will leave the rest of the values as is.

### SpecialCondition

In [28]:
stopdata['SpecialCondition'].unique()

array(['NOT APPLICABLE', 'PRE-EXISTING KNOWLEDGE/INFORMATION',
       'VEHICLE INSPECTION', 'CALL FOR SERVICE', 'SPECIAL DETAIL',
       'DUI CHECKPOINT', 'BOLO'], dtype=object)

All good here, it looks like no cleaning needs to be done for this column.

### Race

In [29]:
stopdata['Race'].unique()

array(['ASIAN/PACIFIC ISLANDER', 'AFRICAN AMERICAN', 'HISPANIC', 'WHITE',
       'OTHER/NOT APPARENT', 'ASIAN OTHER', 'ASIAN INDIAN', 'UNKNOWN',
       'EAST INDIAN', 'HAWAIIAN', 'AM. IND./AL. NATIVE', 'MIXED RACE',
       'FILIPINO', 'GUAMANIAN', 'MIDDLE EASTERN', 'VIETNAMESE',
       'CUBAN/CARRIBEAN', 'NATIVE AMERICAN', 'PACIFIC ISLANDER', 'SAMOAN',
       'HMONG', 'CHINESE', 'ORIENTAL/ASIAN', 'CAMBODIAN', 'JAPANESE'],
      dtype=object)

All good here, it looks like no cleaning needs to be done for this column.

### Gender

In [30]:
stopdata['Gender'].unique()

array(['F', 'M'], dtype=object)

All good here, it looks like no cleaning needs to be done for this column (will need to one hot encode later)

### BirthYear

In [31]:
print(sorted(stopdata['BirthYear'].unique()))

[1900, 1916, 1919, 1921, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2009, 2016, 2017, 2018, 2019]


We know that that in California, the legal age for someone to drive is 15.5 - the idea that some drivers were born in 2016, 2017, 2018, 2019 is highly unlikely (these drivers would be either 0, 1, 2, or 3 based on the years we have data for). In addition, a driver who was born in 1900 would make them the oldest person in the United States, so this value is unlikely as well. Most likely these were mistakes. Because these were most likely all mistakes, we will fill them with the median value. However, lets make this value a little more interpretable to work with - we will change BirthYear to Age

In [32]:
def birthyearfix(data):
    years = [1900, 2016, 2017, 2018, 2019]
    data.loc[data['BirthYear'].isin(years), 'BirthYear'] = np.median(data['BirthYear'])
    
    #change birthyear to age
    data['Age'] = data['StopYear'] - data['BirthYear']
    data = data.loc[:, stopdata.columns != 'BirthYear']
    return data


In [33]:
stopdata = birthyearfix(stopdata)

In [34]:
print(sorted(stopdata['Age'].unique()))

[9.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 97.0, 98.0, 99.0, 101.0]


Great! We fixed our data! While the value of 9 is highly suspicious as well, we will leave it in because it is possible.

### PassengerCount

In [35]:
stopdata['PassengerCount'].value_counts()

0      31046
1      12568
2       3965
3       1671
4        497
5         83
6         20
10        10
7          4
8          3
15         2
30         1
25         1
20         1
425        1
9          1
Name: PassengerCount, dtype: int64

425 passengers seems unlikely, so we will replace this with 0

In [36]:
def passengercountfix(data):
    data.loc[data['PassengerCount'] == 425, 'PassengerCount'] = 0
    return data

In [37]:
stopdata = passengercountfix(stopdata)

### ExitVehicle

In [38]:
stopdata.loc[stopdata['ExitVehicle'].isna()]

,Idx,Citation,EntryDate,StopType,StopDTS,StopCause,ViolationNumber,SpecialCondition,Race,Gender,...,StopYear,StopDayofMonth,StopDayofWeek,StopHour,EntryMonth,EntryYear,EntryDayofMonth,EntryDayofWeek,EntryHour,Age
32,437830,False,2017-07-07 06:36:13+00:00,VEHICLE,2017-07-04 23:22:00+00:00,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,UNK,NOT APPLICABLE,OTHER/NOT APPARENT,M,...,2017,4,1,16,7,2017,7,4,23,31.0
54,437852,False,2017-07-07 10:14:17+00:00,VEHICLE,2017-07-07 06:00:00+00:00,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,22450,NOT APPLICABLE,WHITE,M,...,2017,7,4,23,7,2017,7,4,3,44.0
220,438025,False,2017-07-11 05:40:36+00:00,VEHICLE,2017-07-11 05:27:00+00:00,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,22450,VEHICLE INSPECTION,HISPANIC,M,...,2017,11,1,22,7,2017,11,1,22,22.0
380,438197,False,2017-07-13 17:06:39+00:00,VEHICLE,2017-06-13 23:55:00+00:00,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,23123,NOT APPLICABLE,WHITE,M,...,2017,13,1,16,7,2017,13,3,10,57.0
382,438199,False,2017-07-13 17:11:28+00:00,VEHICLE,2017-06-02 23:16:00+00:00,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,23123,NOT APPLICABLE,WHITE,M,...,2017,2,4,16,7,2017,13,3,10,27.0
490,438310,False,2017-07-14 05:56:59+00:00,VEHICLE,2017-07-14 05:33:00+00:00,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,24601,NOT APPLICABLE,AFRICAN AMERICAN,F,...,2017,14,4,22,7,2017,14,4,22,20.0
572,438400,False,2017-07-15 12:18:33+00:00,VEHICLE,2017-06-26 08:34:00+00:00,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,5200,NOT APPLICABLE,HISPANIC,M,...,2017,26,0,1,7,2017,15,5,5,23.0
581,438409,False,2017-07-16 00:45:19+00:00,VEHICLE,2017-07-16 00:40:00+00:00,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,4000,NOT APPLICABLE,AFRICAN AMERICAN,F,...,2017,16,6,17,7,2017,16,6,17,37.0
648,438481,False,2017-07-17 06:00:56+00:00,VEHICLE,2017-07-17 05:53:00+00:00,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,22450,NOT APPLICABLE,AFRICAN AMERICAN,F,...,2017,17,0,22,7,2017,17,0,23,33.0
770,438603,False,2017-07-19 04:40:07+00:00,VEHICLE,2017-07-19 02:41:00+00:00,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,5200,NOT APPLICABLE,ASIAN/PACIFIC ISLANDER,M,...,2017,19,2,19,7,2017,19,2,21,49.0


In [39]:
stopdata['ExitVehicle'].value_counts()

NO     41500
YES     7756
Name: ExitVehicle, dtype: int64

We will change the null values to the mode, which is NO, and then map this to boolean data - False for "NO" and True for "YES"

In [40]:
def exitvehiclefix(data):
    data.loc[data['ExitVehicle'].isna(), 'ExitVehicle'] = False
    data['ExitVehicle'] = data['ExitVehicle'].replace({'NO': False, 'YES': True})
    return data   

In [41]:
stopdata = stop_data()
stopdata = exitvehiclefix(stopdata)

### ResultCitation

In [42]:
stopdata['ResultCitation'].value_counts()

False    31415
True     18459
Name: ResultCitation, dtype: int64

### Detention Duration

In [ ]:
print(stopdata['DetentionDuration'].value_counts()[:10])

In [44]:
stopdata['ResultReport'].value_counts()

False    48424
True      1450
Name: ResultReport, dtype: int64

In [45]:
stopdata['LimitedLanguage'].value_counts()

False    48728
True      1146
Name: LimitedLanguage, dtype: int64

### Consent Granted and Consent Requested

Consent Granted and Consent Requested

In [46]:
stopdata.loc[stopdata['ConsentRequested'] == 'PAROLE/PROBATION']

,Idx,CitationNumber,EntryDate,StopType,StopDTS,StopCause,ViolationNumber,SpecialCondition,Race,Gender,...,ResultCitation,ResultArrest,ResultReport,ResultAdvised,ResultFC,CadCallNumber,DetentionDuration,OfficerAbra,LimitedLanguage,FID
71,437869,CS0011520,2017-07-07T07:57:00.000Z,VEHICLE,2017-07-07T07:57:00.000Z,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,4000(A)(1),NOT APPLICABLE,AFRICAN AMERICAN,M,...,True,False,False,False,False,17-191590,60,4442,False,2072
4697,436335,CS0137525,2017-06-14T21:22:00.000Z,VEHICLE,2017-06-14T21:22:00.000Z,NOT APPLICABLE,UNKNOWN,NOT APPLICABLE,AFRICAN AMERICAN,F,...,True,False,False,False,False,17-167047,30,3728,False,698
5335,445290,CS0072023,2017-09-21T09:37:00.000Z,VEHICLE,2017-09-21T09:37:00.000Z,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,5200A,NOT APPLICABLE,AFRICAN AMERICAN,M,...,True,False,False,False,False,17-271805,30,4342,False,8336
5749,445711,CS0063042,2017-09-26T00:45:00.000Z,VEHICLE,2017-09-26T00:45:00.000Z,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,4000A,NOT APPLICABLE,AFRICAN AMERICAN,M,...,True,False,True,False,False,17-276591,30,4137,False,8750
8397,451375,CS0177582,2017-11-14T22:16:00.000Z,VEHICLE,2017-11-14T22:16:00.000Z,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,22450,NOT APPLICABLE,HISPANIC,M,...,True,False,True,False,False,17-327991,30,4093,False,13398
12360,446324,CS0063044,2017-10-02T23:40:00.000Z,VEHICLE,2017-10-02T23:40:00.000Z,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,4000A,NOT APPLICABLE,WHITE,F,...,True,False,False,False,False,17-284112,30,4137,False,9361
14129,448093,CS0019122,2017-10-13T17:10:00.000Z,VEHICLE,2017-10-13T17:10:00.000Z,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,14601,PRE-EXISTING KNOWLEDGE/INFORMATION,WHITE,M,...,True,True,True,False,False,17-294738,15,3070,False,10130
14131,448095,CS0019123,2017-10-13T17:50:00.000Z,VEHICLE,2017-10-13T17:50:00.000Z,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,14601,PRE-EXISTING KNOWLEDGE/INFORMATION,HISPANIC,M,...,True,True,True,True,False,17-294712,15,3070,False,10132
14132,448096,CS0019124,2017-10-13T20:36:00.000Z,VEHICLE,2017-10-13T20:36:00.000Z,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,14601,PRE-EXISTING KNOWLEDGE/INFORMATION,WHITE,F,...,True,True,True,True,False,17-294936,15,3070,False,10133
15778,455768,CS0154589,2018-01-05T10:50:00.000Z,VEHICLE,2018-01-05T10:50:00.000Z,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,22350,NOT APPLICABLE,HISPANIC,M,...,True,False,False,False,False,18-004952,30,4282,False,17779


### CadCallNumber

We will change this to a boolean column, which states if the CadCallNumber was one of the top two in the value counts

In [47]:
stopdata['CadCallNumber'].value_counts()[:5]

18-000001    81
17-005000    15
18-000007     7
18-000000     6
18-000011     6
Name: CadCallNumber, dtype: int64

In [48]:
def cadcallnumberfix(data):
    data['CadCallNumber'] = data['CadCallNumber'].map({'18-000001': True, '17-005000': True })
    data.loc[data['CadCallNumber'].isna(), 'CadCallNumber'] = False
    return data

In [49]:
stopdata = cadcallnumberfix(stopdata)

In [50]:
stopdata['CadCallNumber'].value_counts()

False    49778
True        96
Name: CadCallNumber, dtype: int64

### Authority

In [51]:
stopdata['Authority'].value_counts()

NOT APPLICABLE        43888
PAROLE/PROBATION       4236
PROBABLE CAUSE          607
TOW INVENTORY           418
TERRY CURSORY           393
INCIDENT TO ARRE        311
INCIDENT TO ARREST       21
Name: Authority, dtype: int64

"INCIDENT TO ARRE" and "INCIDENT TO ARREST" are the same thing so lets "standardize" this value accordingly.

In [52]:
def authorityfix(data):
    data.loc[data['Authority'] == 'INCIDENT TO ARRE', 'Authority'] = "INCIDENT TO ARREST"
    return data

In [53]:
stopdata = authorityfix(stopdata)

In [54]:
stopdata['Authority'].value_counts()

NOT APPLICABLE        43888
PAROLE/PROBATION       4236
PROBABLE CAUSE          607
TOW INVENTORY           418
TERRY CURSORY           393
INCIDENT TO ARREST      332
Name: Authority, dtype: int64

### Columns we don't need

FID, Idx  have unique values, so we will drop these columns. In addition, the Stoptype column has only one entry, vehicle, so this will be removed as well. Other columns, like the original date columns will be removed as well.

In [55]:
def removecolumns(data):
    cols = [col for col in data.columns if col not in ['FID', 'Idx', 'StopType']]
    data = data[cols]
    return data

In [56]:
stopdata = removecolumns(stopdata)

In [57]:
stopdata.columns

Index(['CitationNumber', 'EntryDate', 'StopDTS', 'StopCause',
       'ViolationNumber', 'SpecialCondition', 'Race', 'Gender', 'BirthYear',
       'ExitVehicle', 'PassengerCount', 'SearchDriver', 'SearchPassenger',
       'SearchVehicle', 'Authority', 'ConsentRequested', 'ConsentGranted',
       'FSWeapons', 'FSDrugs', 'FSCash', 'FSVehicle', 'FSAlcohol',
       'FSOtherProperty', 'FSNothing', 'ResultCitation', 'ResultArrest',
       'ResultReport', 'ResultAdvised', 'ResultFC', 'CadCallNumber',
       'DetentionDuration', 'OfficerAbra', 'LimitedLanguage'],
      dtype='object')

In [58]:
def convert_

SyntaxError: invalid syntax (<ipython-input-58-e97963ec4fd7>, line 1)

In [59]:
def process_data_gm1(data):

    data = (
        data
        # Clean/Tranform Data
        .pipe(citationnumberfix)
        .pipe(convertdatecolumns, 'EntryDate')
        .pipe(convertdatecolumns, 'StopDTS')
        .pipe(violationnumberfix)
        .pipe(birthyearfix)
        .pipe(passengercountfix)
        .pipe(exitvehiclefix)
        .pipe(cadcallnumberfix)
        .pipe(authorityfix)
    
    
        # Remove columns
     
        .pipe(removecolumns)
     
        
     
    ) 
    
    return data

In [60]:
stopdata = stop_data()
stopdata = process_data_gm1(stopdata)
stopdata.head(5)

,Citation,EntryDate,StopDTS,StopCause,ViolationNumber,SpecialCondition,Race,Gender,ExitVehicle,PassengerCount,...,EntryYear,EntryDayofMonth,EntryDayofWeek,EntryHour,StopMonth,StopYear,StopDayofMonth,StopDayofWeek,StopHour,Age
0,False,2017-07-06 05:04:02+00:00,2017-07-06 04:58:00+00:00,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,22108,NOT APPLICABLE,ASIAN/PACIFIC ISLANDER,F,False,1,...,2017,6,3,22,7,2017,6,3,21,24.0
1,False,2017-07-06 05:46:55+00:00,2017-07-06 05:41:00+00:00,MOVING OR HAZARDOUS VIOLATION OF VEHICLE CODE,22450,NOT APPLICABLE,AFRICAN AMERICAN,F,False,0,...,2017,6,3,22,7,2017,6,3,22,23.0
2,False,2017-07-06 06:05:59+00:00,2017-07-06 05:57:00+00:00,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,24616,NOT APPLICABLE,AFRICAN AMERICAN,M,False,1,...,2017,6,3,23,7,2017,6,3,22,28.0
3,False,2017-07-06 06:06:16+00:00,2017-07-06 05:13:00+00:00,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,4000,NOT APPLICABLE,AFRICAN AMERICAN,F,False,0,...,2017,6,3,23,7,2017,6,3,22,48.0
4,False,2017-07-06 06:15:39+00:00,2017-07-06 06:02:00+00:00,REGISTRATION OR REQUIRED EQUIPMENT VIOLATION,24600,NOT APPLICABLE,AFRICAN AMERICAN,M,False,0,...,2017,6,3,23,7,2017,6,3,23,21.0


In [61]:
stopdata.to_csv("cleaned_eda.csv")

## Cleaning for Machine Learning Classification

The pipeline above will make our data perfect for exploring our data and extracting insights through EDA. However, the data above is not the best for training for machine learning- we cannot have categorical features. To fix this, we will do one hot encoding to transform our categorical data.

In [70]:
def onehotencoding(data):
    data['OfficerAbra'] = data['OfficerAbra'].astype(str)
    return pd.get_dummies(data)

In [71]:
def removecolumns2(data):
    cols = [col for col in data.columns if col not in ['FID', 'Idx', 'StopType', 'EntryDate', 'StopDTS']]
    data = data[cols]
    return data

In [72]:
def process_data_gm2(data):

    data = (
        data
        # Clean Data
        .pipe(citationnumberfix)
        .pipe(convertdatecolumns, 'EntryDate')
        .pipe(convertdatecolumns, 'StopDTS')
        .pipe(violationnumberfix)
        .pipe(birthyearfix)
        .pipe(passengercountfix)
        .pipe(exitvehiclefix)
        .pipe(cadcallnumberfix)
        .pipe(authorityfix)
    
        # Remove Columns
     
        .pipe(removecolumns2)
        
        #One Hot Encode
        
        .pipe(onehotencoding)
        
    ) 
    
    return data

In [74]:
stopdata = stop_data()
ml = process_data_gm2(stopdata)
ml.to_csv("ml.csv", index=False)